In [ ]:
# Makes this notebook see the module from one folder above to allow imports from sibling folders!
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import json
from colour import Colour

%matplotlib widget

# params = {"text.usetex" : True,
#           "font.family" : "serif",
#           "font.serif" : ["Computer Modern Serif"],
#           'font.size': 22,}
# mpl.rcParams.update(params)

plt.close('all')

In [ ]:
error_bar_kwargs = {"capsize": 5, "linewidth": 2, "elinewidth": 1, "capthick": 1, "markersize": 10}

def eps2db(epsilon: float) -> float:
    return -10.0 * np.log10(2.0 * np.tanh(epsilon / 2.0))

def db2eps(db_squeezing: float) -> float:
    return 2.0 * np.atanh(np.float_power(10.0, -db_squeezing / 10.0) / 2.0)

def int2tag(n: int, N: int=0) -> str:
    return "{0:0{1}b}".format(n, N)

def tag2int(tag: str) -> int:
    return int(tag, 2)


# Grover error estimate
from scipy.special import erf

# Functions based on B. W. Walshe et al. Streamlined quantum computing with macronode cluster states. Jan. 2022

def analytical_gate_error(db: float, integer: int) -> float:
    # Input quadrature variance
    var = integer * db2eps(db) / 2
    # Success probability rates in each quadrature
    success_rates = erf(np.sqrt(np.pi / (8 * var)))
    return 1 - success_rates

def gate_error_I(db):
    return 1 - (1 - analytical_gate_error(db, 2)) * (1 - analytical_gate_error(db, 2))

def gate_error_P(db):
    return 1 - (1 - analytical_gate_error(db, 2)) * (1 - analytical_gate_error(db, 3))

def grover_with_error_estimate(db) -> float:
    r = (gate_error_I(db) + gate_error_P(db)) / 2
    N = 3 # number of qubits
    k = 2 # number of solutions
    d = 18 # circuit depth

    p_no_err = 1 - 4/3 * r
    p_no_err = p_no_err**(d * N)
    p_err = 1 - p_no_err

    p_success = p_no_err + k / 2**N * p_err

    return p_success

## Plot GKP randomised benchmarking results

In [ ]:
# Retrieve clifford survival fidelity data
paths = [
    "./data/gkp_cliff.dat",
]

samples = []
for path in paths:
    with open(path, 'r') as file:
        samples += json.load(file)

# Sort samples by squeezing
dbs = np.linspace(5, 15, 13)[:2]
fidelities = [[] for _ in range(len(dbs))]
for s in samples:
    i = np.abs(dbs - s["db"]).argmin()
    fidelities[i] += s["fidelities"]

# Compute averages
means = np.mean(fidelities, axis=1)
print(means)
print([len(a) for a in fidelities])

# Plot
plt.figure()
plt.axhline(1/4, color='0', linestyle='-')
for db, fs in zip(dbs, fidelities):
    plt.plot([db]*len(fs), fs, "k*", label="Fidelities")
plt.plot(dbs, means, "ro", label="Averages")

plt.legend()
plt.xlabel("GKP squeezing (dB)")
plt.ylabel("Clifford survival fidelity.")
plt.show()

In [ ]:
from scipy.optimize import curve_fit

def process_rb_samples(samples: list[dict]) -> tuple[dict, dict]:
    # Sort samples by depth
    depths = [s["depth"] for s in samples]
    depths = sorted(set(depths))
    fidelities = []
    purities = []
    for depth in depths:
        fs = []
        ps = []
        for s in samples:
            if not np.isclose(s["depth"], depth):
                continue
            fs.append(s["fidelity"])
            ps.append(s["purity"])
        fidelities.append(fs)
        purities.append(ps)

    print([len(arr) for arr in fidelities])
    fidelity_data = {
        "depths": depths,
        "samples": fidelities,
        "means": [np.mean(arr) for arr in fidelities],
        "errors": [np.std(arr)/np.sqrt(len(arr)) for arr in fidelities],
    }

    purity_data = {
        "depths": depths,
        "samples": purities,
        "means": [np.mean(arr) for arr in purities],
        "sample_std": [np.std(arr) for arr in purities],
    }

    return fidelity_data, purity_data

def fidelity_analysis(data: dict) -> dict:
    depths = data["depths"]
    avg_fidelity = data["means"]
    avg_fidelity_error = data["errors"]
    
    # Fit error rate
    def exp_decay(m, a, p):
        return a * p**m + 1/4
    p_guess = (4 / 3 *(avg_fidelity[-1] - 1/4))**(1/depths[-1])
    initial_guess = [3/4, p_guess]

    popt, pcov = curve_fit(exp_decay, depths, avg_fidelity, sigma=avg_fidelity_error, absolute_sigma=True, p0=initial_guess)

    fit = lambda m: exp_decay(m, *popt)
    p = popt[1]
    p_err = np.sqrt(pcov[1, 1])

    num_qubits = 2 # Fixed at 2
    avg_error_rate = (1-p) * (1 - 2**-num_qubits)
    avg_error_rate_error = p_err * (1 - 2**-num_qubits)

    results = {
        "p": p,
        "p_err": p_err,
        "fit": fit,
        "r": avg_error_rate,
        "r_err": avg_error_rate_error,
    }

    return results 

In [ ]:
# Retrieve RB data
path = "./data/gkp_rb.dat"
with open(path, 'r') as file:
    samples = json.load(file)

# Data analysis

# Sort data by squeezing
dbs = np.linspace(5, 15, 13)
samples_sorted = [[] for _ in range(len(dbs))]

for sample in samples:
    db = sample["db"]
    if db > 12:
        continue
    i = np.abs(dbs - db).argmin()
    samples_sorted[i].append(sample)

# Remove empty data
dbs = dbs.tolist()
i = 0
while i < len(dbs):
    if len(samples_sorted[i]) == 0:
        dbs.pop(i)
        samples_sorted.pop(i)
    else:
        i += 1

processed = [process_rb_samples(samples) for samples in samples_sorted]
fidelity_results = [fidelity_analysis(p[0]) for p in processed]

In [ ]:
# # Plotting each dB individually
# for db, (f, _), r in zip(dbs, processed, fidelity_results, strict=True):
#     temp = r.copy()
#     temp.pop("fit")
#     print(temp)
    
#     ds = f["depths"]
#     plt.figure()
#     plt.plot(ds, f["samples"], "*", color="0.8")
#     plt.errorbar(ds, f["means"], 2 * np.array(f["errors"]), fmt="k.", **error_bar_kwargs)
#     xs = np.linspace(min(ds), max(ds), 100)
#     plt.plot(xs, r["fit"](xs), "r-")
    
#     plt.xlabel("Circuit depth")
#     plt.ylabel("Output fidelity")
#     plt.show()

# Plot fitted functions
plt.figure(figsize=(8, 4))
cmap = mpl.colormaps['grey']
colors = cmap(np.linspace(0, 1, len(dbs)))
for i, db, (f, _), r, c in zip(range(len(dbs)), dbs, processed, fidelity_results, colors, strict=True):
    ds = f["depths"]
    xs = np.linspace(min(ds)-1, 60, 100)
    ys = r["fit"](xs)
    c = "0.5" if i%2==0 else "0.0"
    err_bar = plt.errorbar(ds, f["means"], np.array(f["errors"]), fmt=".", color=c, **error_bar_kwargs)
    fit_line, = plt.plot(xs, ys, "-", color=c)
    
    idx = 5+5*i
    plt.text(xs[idx], ys[idx], "{:.2f} dB".format(db), fontsize=10, va='center', ha='center',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', boxstyle='round,pad=0.2'))
err_bar.set_label("Numerical estimates")
fit_line.set_label("Fit")
plt.xlabel("Circuit depth")
plt.ylabel("Average output fidelity")
# plt.legend()
plt.xlim(0, 65)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()   




# Gate errors
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 5), height_ratios=(3, 1))
error_rate = np.array([e["r"] for e in fidelity_results])
error_rate_std = np.array([e["r_err"] for e in fidelity_results])
# # r^* = sqrt(r)
# # var(r^*) = (dr^*/dr)^2 var(r) = (2 sqrt(r))^-2 var(r)
# gate_error_rate = np.sqrt(error_rate)
# gate_error_rate_std = error_rate_std / (2 * gate_error_rate)

ax1.errorbar(dbs, error_rate, error_rate_std, fmt="k.", **error_bar_kwargs, label="Numerical estimate")
# Analytical
xs = np.linspace(5, 15, 100)
ax1.plot(xs, [gate_error_I(x) for x in xs], "-", color="0.5", label="Analytical estimate")
ax1.plot(xs, [gate_error_P(x) for x in xs], "-", color="0.5")
ax1.plot(xs, [(gate_error_I(x) + gate_error_P(x))/2 for x in xs], "--", color="0.5", label="Mean of analytics")

ax1.set_yscale('log')
ax1.set_xlim(5.5, 12.5)
ax1.set_ylim(1.5e-4, 1.1)
ax1.set_ylabel("Average gate error rate")
ax1.grid(which="both")
ax1.legend()

# Normalised residuals
mean = np.array([(gate_error_I(db) + gate_error_P(db))/2 for db in dbs])
residuals = (error_rate - mean) / error_rate_std
ax2.plot(dbs, residuals, "k*")
ax2.axhline(0, color="0.5", linestyle="--")
ax2.set_ylim(-3.5, 3.5)
ax2.set_ylabel("Normalised residuals")
ax2.set_xlabel("GKP squeezing (dB)")
ax2.grid(axis="x")

plt.tight_layout()
plt.show()






# # Plot logical purities
# plt.figure()
# for db, (_, p), c in zip(dbs, processed, colors, strict=True):
#     ds = p["depths"]
#     xs = np.linspace(min(ds), max(ds), 100)
#     plt.errorbar(ds, p["means"], np.array(p["sample_std"]), fmt=".", color=c, label="{:.2f} dB".format(db), **error_bar_kwargs)
# plt.xlabel("Circuit depth")
# plt.ylabel("Output fidelity")
# plt.legend()
# plt.grid()
# plt.show()

## Plot GKP grover simulation data

In [ ]:
# Retreive simulation data
N = 3
solutions = [(0, 4), (2, 7), (3, 6)]
paths = [
    "./data/gkp_grover_04.dat",
    "./data/gkp_grover_27.dat",
    "./data/gkp_grover_36.dat",
]

entries = []
for path in paths:
    with open(path, 'r') as file:
        entries.append(json.load(file))

In [ ]:
# # Pretty print runs (for debugging)
# for entry in entries[0]:
#     db = round(eps2db(entry["epsilon"]), 4)
#     print(f"Sim with {db}dB of squeezing.")
#     for i, p in enumerate(np.diag(entry["rho_real"])):
#         string = int2tag(i, 3)
#         string = f"{string} : {p:1.4f}"
#         if i in [2, 7]:
#             string = Colour.colour_this(string, Colour.OKGREEN, Colour.BOLD)
#         elif p > 0.1:
#             string = Colour.colour_this(string, Colour.FAIL, Colour.BOLD)
#         print(string)
#     print()

In [ ]:
# Process data
processed = []

dbs = np.linspace(5, 15, 13)[2:-1]  # remove less interesting regimes
for entry_sol, entry_data in zip(solutions, entries):
    # dbs = sorted(set(eps2db(e["epsilon"]) for e in entry_data))
    
    success = [[] for _ in range(len(dbs))]
    for entry in entry_data:
        db = eps2db(entry["epsilon"])
        i = np.abs(dbs - db).argmin()
        if abs(db - dbs[i]) > 1e-6:
            continue
        ps = np.diag(entry["rho_real"])
        p = sum(ps[n] for n in entry_sol)
        success[i].append(p)
    
    averages = [np.mean(ps) if ps else float('NaN') for ps in success]
    errors = [2*np.std(ps)/np.sqrt(len(ps)) if ps else float('NaN') for ps in success]
    processed.append({
        "sols": entry_sol,
        "dbs": dbs,
        "ps": success,
        "avgs": averages,
        "errs": errors,
    })

# Combine entries into a single data set
combined = {
    "dbs": dbs,
    "ps": [[] for _ in range(len(dbs))],
    "avgs": [],
    "errs": [],
}
for entry in processed:
    for i in range(len(combined["dbs"])):
        combined["ps"][i] += entry["ps"][i]
combined["avgs"] = [np.mean(ps) for ps in combined["ps"]]
combined["errs"] = [2*np.std(ps)/np.sqrt(len(ps)) for ps in combined["ps"]]

# # Remove certain undersampled dBs
# for key in combined.keys():
#     combined[key] = combined[key][2:]


In [ ]:
# Plot separate runs
colors = ["0.0", "0.25", "0.5"]
plt.figure(figsize=(8, 4))
plt.axhline(13/28, color='0', linestyle='-')
plt.axhline(2/8, color='0', linestyle='--')

# # Plot samples
# for entry, c in zip(processed, colors, strict=True):
#     xs = entry["dbs"]
#     ys = entry["ps"]
#     for x0, y in zip(xs, ys):
#         x = np.repeat([x0], len(y))
#         plt.plot(x, y, c+"*", alpha=.1)
# Plot averages and errors
for entry, c in zip(processed, colors, strict=True):
    tags = [int2tag(n, N) for n in entry["sols"]]
    label = "Oracle: " + ", ".join(tags)
    averages = entry["avgs"]
    errors = entry["errs"]
    xs = entry["dbs"]
    plt.errorbar(xs, averages, errors, fmt=".-", color=c, label=label, **error_bar_kwargs)

plt.legend()
plt.xlabel("GKP squeezing (dB)")
plt.ylabel("Success probability")
plt.ylim(-0.05, 1.05)
plt.grid()
plt.tight_layout()
plt.show()


# Print combined data
print("Sampled dBs:", np.round(combined["dbs"], 3).tolist())
print("Number of samples:", [len(ls) for ls in combined["ps"]])

plt.figure(figsize=(8, 4))
plt.axhline(13/28, color='0', linestyle='-')
plt.axhline(2/8, color='0', linestyle='--')

xs = combined["dbs"]
ys = combined["ps"]
# for x0, y in zip(xs, ys):
#     x = np.repeat([x0], len(y))
#     obj, = plt.plot(x, y, "*", color="0.8")
# obj.set_label("Samples")
averages = combined["avgs"]
errors = combined["errs"]
plt.errorbar(xs, averages, errors, fmt="k.", **error_bar_kwargs, label="Numerical data")

xs = np.linspace(dbs[0], dbs[-1], 100)
plt.plot(xs, [grover_with_error_estimate(x) for x in xs], "k-", label="RB estimate")

plt.legend()
plt.xlabel("GKP squeezing (dB)")
plt.ylabel("Success probability")
plt.ylim(-0.05, 1.05)
plt.grid()
plt.tight_layout()
plt.show()